In [482]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [483]:
using Plots

In [ ]:
gmsh.open("transient.geo")

In [485]:
#gmsh.fltk.run()

In [ ]:
body = FEM.material("body")
problem = FEM.Problem([body], type="PlaneHeatConduction")

In [ ]:
K = FEM.heatConductionMatrix(problem)

In [ ]:
C = FEM.heatCapacityMatrix(problem, lumped=false)

In [ ]:
left = FEM.temperatureConstraint("left", T=0)
right = FEM.temperatureConstraint("right", T=100)
hf = FEM.heatFlux("left", qn=0)

In [ ]:
q0 = zeros(problem.non)

In [491]:
FEM.applyBoundaryConditions!(problem, K, C, q0, [left, right])

In [ ]:
T0 = FEM.initialTemperature(problem, "right", T=100)

In [ ]:
λₘᵢₙ = FEM.smallestEigenValue(K, C)

In [ ]:
λₘₐₓ = FEM.largestEigenValue(K, C)

In [ ]:
ϑ = 0.4
Δt = 2 / ((1 - 2ϑ) * λₘₐₓ)
ϑ = 0.5

In [ ]:
xy = FEM.FDMaccuracyAnalysis(λₘᵢₙ, λₘₐₓ, Δt, ϑ=ϑ)
plot(xy)

In [ ]:
T, t = FEM.FDM(K, C, q0, T0, 1000 * Δt, Δt, ϑ=ϑ)

In [499]:
T1 = FEM.showDoFResults(problem, T, "T", t=t, visible=true);

In [ ]:
p1, xy = FEM.plotOnPath(problem, "path", T1, plot=true)
plot(xy)

In [ ]:
gmsh.fltk.run()

In [502]:
gmsh.finalize()